In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext watermark
%watermark

Last updated: 2023-11-24T08:23:25.929605+05:30

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [17]:
# Either you can store the  OpenAI key in the “OPENAI_API_KEY” environment variable.
# or pass it here as below from a config.ini
import configparser
workingFolder='C:\\Users\\jfrancis\\AI Journey\\Gen AI\\'
# Read the configuration file
config = configparser.ConfigParser()
config.read(workingFolder+'\\config.ini')
OPENAI_API_KEY=config.get('General','OPENAI_API_KEY')
ACTIVELOOP_TOKEN=config.get('General','ACTIVELOOP_TOKEN')
ACTIVELOOP_ORG_ID=config.get('General','ACTIVELOOP_ORG_ID')
HUGGINGFACEHUB_API_TOKEN=config.get('General','HUGGINGFACEHUB_API_TOKEN')
GOOGLE_API_KEY=config.get('General','GOOGLE_API_KEY')
GOOGLE_CSE_ID=config.get('General','GOOGLE_CSE_ID')
COHERE_API_KEY=config.get('General','COHERE_API_KEY')
WOLFRAM_ALPHA_APPID=config.get('General','WOLFRAM_ALPHA_APPID')

In [21]:
import langchain
import langchain_experimental
import requests
import openai
import wikipedia
import wolframalpha
%watermark --iversions 

requests              : 2.31.0
wikipedia             : 1.4.0
openai                : 1.3.2
langchain             : 0.0.336
wolframalpha          : 5.0.0
langchain_experimental: 0.0.42



## LangChain's Tool Utilization: Examining Diverse Applications through Illustrative Scenarios 

### Introduction

Tools are modular, reusable components meticulously designed to accomplish specific tasks or provide answers to distinct types of questions. By integrating these tools seamlessly into the system, users can effortlessly tap into a diverse range of functionalities and information sources to tackle challenges and generate meaningful responses. In this lesson, we will explore the various Tools in LangChain, uncovering their unique capabilities.

A few notable examples of tools in LangChain, without getting into technical details, are:

    Google Search: This tool uses the Google Search API to fetch relevant information from the web, which can be used to answer queries related to current events, facts, or any topic where a quick search can provide accurate results.
    Requests: This tool employs the popular Python library "requests" to interact with web services, access APIs, or obtain data from different online sources. It can be particularly useful for gathering structured data or specific information from a web service.
    Python REPL: The Python REPL (Read-Eval-Print Loop) tool allows users to execute Python code on-the-fly to perform calculations, manipulate data, or test algorithms. It serves as an interactive programming environment within the LangChain system.
    Wikipedia: The Wikipedia tool leverages the Wikipedia API to search and retrieve relevant articles, summaries, or specific information from the vast repository of knowledge on the Wikipedia platform.
    Wolfram Alpha: With this tool, users can tap into the powerful computational knowledge engine of Wolfram Alpha to answer complex questions, perform advanced calculations, or generate visual representations of data.
    
### Google-Search

LLMs inherently possess knowledge only up to the point at which they were trained, leaving them unaware of any information beyond that timeframe. Integrating search engines as tools within the LangChain system presents a significant advantage. The LangChain library provides a Google Search API wrapper that you can use within your project. You can use this wrapper as a standalone utility or as a tool within an agent.

First, make sure you have an API Key and Custom Search Engine ID for the Google Search API. If you don’t already have a Custom Search Engine ID, the following tutorial is a helpful guide for generating one. https://help.elfsight.com/article/331-how-to-get-search-engine-id

Also, getting Google Search API is straightforward. If you have Google Cloud Platform access, you’d just go to the credentials page and click Create credentials > API key. https://support.google.com/googleapi/answer/6158862?hl=en

Set them as environment variables like the following. Then, you can use GoogleSearchAPIWrapper to receive k top search results given a query. 

In [18]:
# Get the token from Hugginface/Active loop website before this. Now we are taking from the config.ini
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
os.environ['GOOGLE_CSE_ID'] = GOOGLE_CSE_ID
os.environ['WOLFRAM_ALPHA_APPID'] = WOLFRAM_ALPHA_APPID

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
os.environ['ACTIVELOOP_TOKEN'] = ACTIVELOOP_TOKEN
my_activeloop_org_id = ACTIVELOOP_ORG_ID

In [5]:
# As a standalone utility:
from langchain. utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
search.results("What is the capital of Spain?", 3)

[{'title': 'Madrid, Spain - Image of the Week - Earth Watching',
  'link': 'https://earth.esa.int/web/earth-watching/image-of-the-week/content/-/article/madrid-spain/',
  'snippet': 'Madrid is the capital and largest city of Spain. The population of the city is roughly 3.3 million and the entire population of the Madrid metropolitan area\xa0...'},
 {'title': 'Madrid | History, Population, Climate, & Facts | Britannica',
  'link': 'https://www.britannica.com/place/Madrid',
  'snippet': "4 days ago ... Madrid, city, capital of Spain and of Madrid provincia (province). Spain's arts and financial centre, the city proper and province form a\xa0..."},
 {'title': 'Madrid - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/Madrid',
  'snippet': 'Madrid lies on the River Manzanares in the central part of the Iberian Peninsula at about 650 metres above mean sea level. Capital city of both Spain and the\xa0...'}]

In the LangChain library, using the available tools requires some necessary steps to be taken. First, you need to initialize an agent, which is the central manager for effectively using these tools. Then, we need to define the language model that we want the agent to use

In [6]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003", temperature=0)

Now, we can initialize an agent and load the google-search tool for it to use. The agent will load the search results and provide them to the llm to answer our question. The ZERO_SHOT_REACT_DESCRIPTION type gives the freedom to choose any of the defined tools to provide context for the model based on their description. 

In [7]:
from langchain.agents import initialize_agent, load_tools, AgentType

tools = load_tools(["google-search"])

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

print( agent("What is the national drink in Spain?") )



> Entering new AgentExecutor chain...
 I should research this online.
Action: google_search
Action Input: "national drink in Spain"
Observation: Sangría is Spain's national drink, so of course it belongs on this list! Spain is famous for it's wine, which is the base of this drink. Lots of fresh fruit ... Although there is nothing officially considered the national alcoholic drink of Spain, many people would say is the Spanish sherry wine (also known as vino de ... Apr 20, 2021 ... Cava is the champagne of Spain and it comes in the form of a light, bubbly wine available in both white and rosé. Cava is often consumed ... "Very Old Rare Sherry", Pedro Ximenez by Garvey. Jerez de la Frontera (Andalusia, Spain), aged 30 years. Sherry is a national liquor of Spain. Nov 27, 2012 ... ... national drink (and that ampersands are expendable). From Cadiz in the south to San Sebastian in the north, the gin tonic is the undisputed ... Cava is Spain's version of Champagne. It's made in a very simil

### Requests

The internet is an extensive source of information that Large Language Models cannot access directly. LangChain offers a convenient wrapper built around the Python Requests module to facilitate seamless interaction between LLMs and this wealth of information. This wrapper accepts a URL as an input and efficiently retrieves data from the specified URL, allowing LLMs to obtain and process web-based content effortlessly.

In this example, we’ll set up a  fake RESTful backend using mockapi.io. To do it, follow these steps:

    Go to mockapi.io and sign up for a free account.
    After signing up, log in to your account.
    Click on "New Project" (the "+" icon) and give your project a name. You don't need to fill in any optional fields.
    Once the project is created, click on it to view your unique API endpoint.
    Click on "New Resource" to create a new resource for your API. For example, if you want to create an endpoint for users, you can name the resource "users.”
    Define the schema for your resource. For instance, if you want each user to have an id, name, and email, you can use the following schema:

Click on the "Create" button to create the resource with the defined schema.

This fake backend will have an endpoint to retrieve information about fake users stored in the backend. A dictionary will represent each user. For instance:

Let's use the LangChain tools to interact with our fake RESTful backend. First, import the necessary libraries and initialize the agent with the desired tools. Then, ask the agent to do an HTTP call at “https://644696c1ee791e1e2903b0bb.mockapi.io/user”: this is the address of our specific mockapi instance, where you should find 30 users. If you want to try your mockapi instance, then replace the address with “https://<YOUR-MOCKAPI-SERVER-ID>.mockapi.io/user”.

In [8]:
from langchain.agents import AgentType

tools = load_tools(["requests_all"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
  )

response = agent.run("Get the list of users at https://644696c1ee791e1e2903b0bb.mockapi.io/user and tell me the total number of users")



> Entering new AgentExecutor chain...
 I need to get the list of users from the URL
Action: requests_get
Action Input: https://644696c1ee791e1e2903b0bb.mockapi.io/user
Observation: [{"createdAt":"2023-04-24T07:55:47.634Z","name":"Mr. Kelly Balistreri","avatar":"https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/1244.jpg","id":"1"},{"createdAt":"2023-04-24T03:54:44.108Z","name":"Bradley Cronin","avatar":"https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/615.jpg","id":"2"},{"createdAt":"2023-04-24T14:32:29.991Z","name":"Jennifer Block Sr.","avatar":"https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/105.jpg","id":"3"},{"createdAt":"2023-04-23T23:30:05.868Z","name":"Nathaniel Toy V","avatar":"https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/533.jpg","id":"4"},{"createdAt":"2023-04-24T03:01:15.797Z","name":"Regina Veum","avatar":"https://cloud

As seen in this example, the agent, initialized with the Request tool and the OpenAI language model, processes the given prompt. It identifies the need to fetch data from the provided URL using a GET request, which is facilitated by the tool of the request. Upon retrieving the user data, the agent analyzes the number of users and returns the result, completing the task.

### Python-REPL

Another tool feature in LangChain is the Python REPL tool, which allows you to execute Python code generated by the language model. This can be useful for complex calculations where the language model generates code to calculate the answer since LLMs are not good at solving algorithmic and math problems. 

Here’s an example of  Python-REPL tool usage:

In [15]:
from langchain_experimental.utilities import PythonREPL
from langchain.agents import Tool

python_repl = PythonREPL()

# You can create the tool to pass to an agent
tools = [
    Tool(
        name="python_repl",
        description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
        func=python_repl.run,
    )
]
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
  )

print( agent.run("Create a list of random strings containing 4 letters, list should contain 30 examples, and sort the list alphabetically") )



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I need to generate a list of random strings, sort them, and then print the result
Action: python_repl
Action Input: 
import random

my_list = []
for i in range(30):
    my_list.append(''.join(random.choices(string.ascii_lowercase, k=4)))

my_list.sort()
print(my_list)
Observation: NameError("name 'string' is not defined")
Thought: I need to import the string module
Action: python_repl
Action Input: 
import random
import string

my_list = []
for i in range(30):
    my_list.append(''.join(random.choices(string.ascii_lowercase, k=4)))

my_list.sort()
print(my_list)
Observation: ['amlb', 'bajh', 'dkbm', 'emau', 'ezqv', 'hbls', 'hken', 'iisv', 'jhso', 'jprg', 'lmzb', 'lsgo', 'lxyr', 'ndya', 'nlxb', 'owtl', 'qwma', 'rnbt', 'syob', 'tuju', 'uwpa', 'vajw', 'vhea', 'vwod', 'xkqi', 'xkqz', 'xmgh', 'ysjm', 'yssx', 'zjnn']

Thought: I now know the final answer
Final Answer: ['amlb', 'bajh', 'dkbm', 'emau', 'ezqv', 'hbls', 'hken', 'iisv', 'jhso', 'jprg', 'lmzb', 'lsgo', 'lxyr', 'ndya', 'nlxb', 'ow

### Wikipedia

The Wikipedia API tool in LangChain is a powerful tool that allows language models to interact with the Wikipedia API to fetch information and use it to answer questions. 

In [16]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

tools = load_tools(["wikipedia"])

print( agent.run("What is Nostradamus know for") )



> Entering new AgentExecutor chain...
 He is a famous prophet
Action: python_repl
Action Input: print("Nostradamus is known for his prophecies and predictions.")
Observation: Nostradamus is known for his prophecies and predictions.

Thought: I now know the final answer
Final Answer: Nostradamus is known for his prophecies and predictions.

> Finished chain.
Nostradamus is known for his prophecies and predictions.


### Wolfram-Alpha

In LangChain, you can integrate Wolfram Alpha by using the WolframAlphaAPIWrapper utility. First, you need to set up a Wolfram Alpha developer account and get your APP ID. https://products.wolframalpha.com/api

In [20]:
from langchain. utilities.wolfram_alpha import WolframAlphaAPIWrapper

wolfram = WolframAlphaAPIWrapper()
result = wolfram.run("What is 2x+5 = -3x + 7?")
print(result)  # Output: 'x = 2/5'

Assumption: 2 x + 5 = -3 x + 7 
Answer: x = 2/5


Utilizing the WolframAlphaAPIWrapper from the LangChain library, this code solves the algebraic equation. Let’s observe the WolframAlpha as a tool:

In [22]:
tools = load_tools(["wolfram-alpha"])

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

print( agent.run("How many days until the next Solar eclipse") )



> Entering new AgentExecutor chain...
 I need to find out when the next Solar eclipse is
Action: wolfram_alpha
Action Input: When is the next Solar eclipse
Observation: Assumption: next | solar eclipse | after today 
Answer: Monday, April 8, 2024 (4 months from now)
Thought: I now know the final answer
Final Answer: 4 months from now, on Monday, April 8, 2024.

> Finished chain.
4 months from now, on Monday, April 8, 2024.


### A Showcase of Resourceful Collaboration

Agents have the capability to employ a variety of tools in order to provide comprehensive and accurate responses to complex inquiries. These agents are better equipped to deliver well-rounded solutions by integrating resources such as Wikipedia and Wolfram-Alpha. Here’s an example of it:

In [ ]:
tools = load_tools(["wolfram-alpha", "wikipedia"], llm=llm)

agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")